In [1]:
import threading
import queue
import time
import sounddevice as sd
import numpy as np
import torch
from transformers import AutoTokenizer
from parler_tts import ParlerTTSForConditionalGeneration
from openai import OpenAI

# === YOUR API CLIENT ===
client = OpenAI(
    base_url="https://api.sambanova.ai/v1",
    api_key="4a81daa9-5f3d-409b-9f30-ebedb379219a"
)

# === Device & Models ===
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# TTS setup
TOKENIZER = AutoTokenizer.from_pretrained("ai4bharat/indic-parler-tts")
MODEL_1   = ParlerTTSForConditionalGeneration.from_pretrained("ai4bharat/indic-parler-tts").to(DEVICE)
MODEL_2   = ParlerTTSForConditionalGeneration.from_pretrained("ai4bharat/indic-parler-tts").to(DEVICE)

# Voice conditioning
DESC_TOKENIZER = AutoTokenizer.from_pretrained(MODEL_1.config.text_encoder._name_or_path)
DESCRIPTION    = "Rohit's voice is monotone yet slightly fast in delivery, with minimal background noise."
DESC_INPUTS    = DESC_TOKENIZER(DESCRIPTION, return_tensors="pt").to(DEVICE)
DESC_INPUT_IDS = DESC_INPUTS.input_ids
DESC_ATTN_MASK = DESC_INPUTS.attention_mask

SAMPLING_RATE = MODEL_1.config.sampling_rate
sd.default.latency = 'low'

# === TTS Worker ===
def tts_worker(text_queue, audio_dict, model, audio_events):
    while True:
        item = text_queue.get()
        if item is None:
            break
        seq_id, chunk, start_time = item

        inp = TOKENIZER(chunk, return_tensors="pt").to(DEVICE)
        with torch.no_grad():
            aud = model.generate(
                input_ids=DESC_INPUT_IDS,
                attention_mask=DESC_ATTN_MASK,
                prompt_input_ids=inp.input_ids,
                prompt_attention_mask=inp.attention_mask
            )
        audio = aud.cpu().numpy().squeeze()
        if audio.ndim>1: audio = audio.flatten()
        audio = audio.astype(np.float32)
        audio /= np.max(np.abs(audio)) or 1.0

        audio_dict[seq_id] = (audio, start_time)
        audio_events[seq_id].set()

# === Playback Thread ===
def playback_thread_fn(audio_dict, audio_events, chunk_ids, done_stream):
    next_seq = 0
    while True:
        while next_seq >= len(chunk_ids):
            if done_stream.is_set(): break
            time.sleep(0.01)
        if next_seq >= len(chunk_ids) and done_stream.is_set():
            break

        audio_events[next_seq].wait()
        audio, st = audio_dict[next_seq]
        if next_seq==0:
            latency = time.time() - st
            print(f"\n Latency: {latency:.2f}s")
        sd.play(audio, samplerate=SAMPLING_RATE)
        sd.wait()
        next_seq += 1

    print("All done.")

# === Streaming + Dynamic Chunking + TTS ===
def stream_llm_tts(prompt, initial_size=10, max_size=1000):
    text_q1, text_q2 = queue.Queue(), queue.Queue()
    audio_dict, audio_events = {}, {}
    chunk_ids = []
    lock = threading.Lock()
    done_stream = threading.Event()

    # start TTS threads
    t1 = threading.Thread(target=tts_worker, args=(text_q1, audio_dict, MODEL_1, audio_events), daemon=True)
    t2 = threading.Thread(target=tts_worker, args=(text_q2, audio_dict, MODEL_2, audio_events), daemon=True)
    t1.start(); t2.start()
    # start playback
    pb = threading.Thread(target=playback_thread_fn, args=(audio_dict, audio_events, chunk_ids, done_stream), daemon=True)
    pb.start()

    buffer = ""
    toggle = 0
    sizes  = [initial_size, initial_size]

    response = client.chat.completions.create(
        model="Meta-Llama-3.2-3B-Instruct",
        messages=[
            {"role":"system", "content":"You are a Hindi-only assistant. Keep responses short."},
            {"role":"user",   "content":prompt}
        ],
        stream=True
    )

    # stream tokens
    for chunk in response:
        delta = chunk.choices[0].delta
        if hasattr(delta, "content") and delta.content:
            # print text live
            print(delta.content, end="", flush=True)

            buffer += delta.content
            # emit full chunks
            while len(buffer) >= sizes[toggle % 2]:
                txt = buffer[:sizes[toggle % 2]].strip()
                buffer = buffer[sizes[toggle % 2]:]

                with lock:
                    seq = len(chunk_ids)
                    chunk_ids.append(seq)
                    audio_events[seq] = threading.Event()

                # dispatch chunk
                (text_q1 if seq % 2 == 0 else text_q2).put((seq, txt, time.time()))

                # update sizes
                sizes[toggle % 2] = min(sizes[toggle % 2] * 2, max_size)
                toggle += 1

    # leftover
    if buffer.strip():
        print(buffer, end="", flush=True)
        with lock:
            seq = len(chunk_ids)
            chunk_ids.append(seq)
            audio_events[seq] = threading.Event()
        (text_q1 if seq % 2 == 0 else text_q2).put((seq, buffer.strip(), time.time()))

    # signal end
    text_q1.put(None); text_q2.put(None)
    done_stream.set()

    t1.join(); t2.join(); pb.join()





Flash attention 2 is not installed
ParlerTTSForConditionalGeneration has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
ParlerTTSForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten.

In [2]:
prompt = "Tell me a short story" 
stream_llm_tts(prompt, initial_size=20, max_size=200)

एक छोटे से गाँव में एक लड़का रहता था। वह एक दिन एक पुरानी किताब में एक रहस्यमय पत्र पढ़ा। वह पत्र उसे एक पुराने मंदिर में ले जाता था। वहां, वह एक सुंदर स्त्री को देखता है जो उसके भविष्य को बताती है।ं ले जाता था। वहां, वह एक सुंदर स्त्री को देखता है जो उसके भविष्य को बताती है।

`prompt_attention_mask` is specified but `attention_mask` is not. A full `attention_mask` will be created. Make sure this is the intended behaviour.
`prompt_attention_mask` is specified but `attention_mask` is not. A full `attention_mask` will be created. Make sure this is the intended behaviour.



[play #0] Latency: 1.96s
All done.

Finished streaming & playback.


In [ ]:
from RealtimeSTT import AudioToTextRecorder
print("Wait until it says 'speak now'")
import time

recorder = AudioToTextRecorder(model="medium",print_transcription_time=True,language="hi",spinner=False)
def pr(text):
    print(text)
    
while True:
    # recorder.text(speak_from_prompt)
    recorder.text(stream_llm_tts)
    #small, large
    

In [1]:
!python model.py


Flash attention 2 is not installed
ParlerTTSForConditionalGeneration has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
ParlerTTSForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten.